In [ ]:
import pandas
import numpy
import re
import nltk
import pickle
import joblib
from nltk.corpus import stopwords
nltk.download('stopwords')
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers import Conv1D
from keras.regularizers import l1
from keras.regularizers import l2
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Dataset of Twitter Reviews



In [ ]:
twitter_reviews = pandas.read_csv("/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/IMDB Dataset.csv")
twitter_reviews.isnull().values.any()
print(twitter_reviews.head())


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


# Preprocessing Of Data


In [ ]:
TAG_REMOVAL = re.compile(r'<[^>]+>')
def tags_removal(txt):
    return TAG_REMOVAL.sub('', txt)
    
def preprocess(sentence):
    sentence_of_words = tags_removal(sentence)
    sentence_of_words = re.sub('[^a-zA-Z]', ' ', sentence_of_words)
    sentence_of_words = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence_of_words)
    sentence_of_words = re.sub(r'\s+', ' ', sentence_of_words)
    return sentence_of_words

In [ ]:
Y = twitter_reviews['sentiment']
Y = numpy.array(list(map(lambda x: 1 if x == 'positive' else 0, Y)))
sentences = list(twitter_reviews['review'])
X = []  
for sen in sentences:
    X.append(preprocess(sen)) 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
vocabsize = 1
vocabsize += len(tokenizer.word_index)
maxlength = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlength)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlength)

Using the GloVe precomputed Word Vectors

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
glove_pretrained = open('/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/glove.6B.100d.txt', encoding="utf8")
embeddings_dict = dict()

for line in glove_pretrained:
    sen = line.split()
    word = sen[0]
    vector = asarray(sen[1 : ], dtype='float32')
    embeddings_dict[word] = vector
glove_pretrained.close()

In [ ]:
all_words = tokenizer.word_index.items()
embedding_matrix = zeros((vocabsize, 100))
for word, index in all_words:
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[index]=embedding_vector

MLP + TRAIN EMBEDDING


In [ ]:
model_mlp = Sequential()
embedding_layer = Embedding(vocabsize, 100, input_length=maxlength)
model_mlp.add(embedding_layer)
model_mlp.add(Flatten())
model_mlp.add(Dense(16, activation='sigmoid'))
model_mlp.add(Dense(1, activation='sigmoid'))
model_mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model_mlp.summary())






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          9254700   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 9,414,733
Trainable params: 9,414,733
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history_mlp = model_mlp.fit(X_train, Y_train, batch_size=32000, epochs=100, verbose=1, validation_split=0.2)
score = model_mlp.evaluate(X_test, Y_test, verbose=1)
scoretrain = model_mlp.evaluate(X_train, Y_train, verbose=1)
print('Train Accuracy:', scoretrain[1])
print('Test Accuracy:', score[1])




Train on 32000 samples, validate on 8000 samples
Epoch 1/100





32000/32000 [==============================] - 5s 143us/step - loss: 0.7151 - acc: 0.5012 - val_loss: 0.7114 - val_acc: 0.5000
Epoch 2/100
32000/32000 [==============================] - 0s 2us/step - loss: 0.7104 - acc: 0.5012 - val_loss: 0.7071 - val_acc: 0.5000
Epoch 3/100
32000/32000 [==============================] - 0s 2us/step - loss: 0.7058 - acc: 0.5012 - val_loss: 0.7027 - val_acc: 0.5000
Epoch 4/100
32000/32000 [==============================] - 0s 2us/step - loss: 0.7010 - acc: 0.5012 - val_loss: 0.6981 - val_acc: 0.5104
Epoch 5/100
32000/32000 [==============================] - 0s 2us/step - loss: 0.6961 - acc: 0.5122 - val_loss: 0.6936 - val_acc: 0.5119
Epoch 6/100
32000/32000 [==============================] - 0s 2us/step - loss: 0.6913 - acc: 0.5141 - val_loss: 0.6895 - val_acc: 0.5128
Epoch 7/100
32000/32000 [==============================] - 0s 2us/step - loss: 0.6868 - acc: 0.5174 - val_loss: 0.6860 

MLP + GLOVE


In [ ]:
model_mlp_glove = Sequential()
embedding_layer = Embedding(vocabsize, 100, weights=[embedding_matrix], input_length=maxlength, trainable=False)
model_mlp_glove.add(embedding_layer)
model_mlp_glove.add(Flatten())
model_mlp_glove.add(Dense(16, activation='sigmoid'))
model_mlp_glove.add(Dense(1, activation='sigmoid'))
model_mlp_glove.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model_mlp_glove.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          9254700   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 9,414,733
Trainable params: 160,033
Non-trainable params: 9,254,700
_________________________________________________________________
None


In [ ]:
history_mlp_glove = model_mlp_glove.fit(X_train, Y_train, batch_size=32000, epochs=250, verbose=1, validation_split=0.2)
score = model_mlp_glove.evaluate(X_test, Y_test, verbose=1)
scoretrain = model_mlp_glove.evaluate(X_train, Y_train, verbose=1)
print('Train Accuracy:', scoretrain[1])
print('Test Accuracy:', score[1])

Train on 32000 samples, validate on 8000 samples
Epoch 1/250
32000/32000 [==============================] - 0s 8us/step - loss: 0.6964 - acc: 0.5041 - val_loss: 0.7436 - val_acc: 0.5005
Epoch 2/250
32000/32000 [==============================] - 0s 1us/step - loss: 0.7424 - acc: 0.4988 - val_loss: 0.7386 - val_acc: 0.5038
Epoch 3/250
32000/32000 [==============================] - 0s 1us/step - loss: 0.7360 - acc: 0.5044 - val_loss: 0.7071 - val_acc: 0.5102
Epoch 4/250
32000/32000 [==============================] - 0s 1us/step - loss: 0.7043 - acc: 0.5117 - val_loss: 0.6656 - val_acc: 0.6327
Epoch 5/250
32000/32000 [==============================] - 0s 1us/step - loss: 0.6629 - acc: 0.6364 - val_loss: 0.6865 - val_acc: 0.5130
Epoch 6/250
32000/32000 [==============================] - 0s 1us/step - loss: 0.6842 - acc: 0.5138 - val_loss: 0.6851 - val_acc: 0.5145
Epoch 7/250
32000/32000 [==============================] - 0s 1us/step - loss: 0.6827 - acc: 0.5142 - val_loss: 0.6642 - val_acc:

CNN + Trainable Embedding

In [ ]:
model_cnn = Sequential()
embedding_layer = Embedding(vocabsize, 100, input_length=maxlength)
model_cnn.add(embedding_layer)
model_cnn.add(Conv1D(512, 5, activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(16, activation='sigmoid'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model_cnn.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          9254700   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 512)           256512    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                8208      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 9,519,437
Trainable params: 9,519,437
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history_cnn = model_cnn.fit(X_train, Y_train, batch_size=16000, epochs=35, verbose=1, validation_split=0.2)
score = model_cnn.evaluate(X_test, Y_test, verbose=1)
scoretrain = model_cnn.evaluate(X_train, Y_train, verbose=1)
print('Train Accuracy:', scoretrain[1])
print('Test Accuracy:', score[1])

Train on 32000 samples, validate on 8000 samples
Epoch 1/35
32000/32000 [==============================] - 6s 196us/step - loss: 0.6992 - acc: 0.5012 - val_loss: 0.6951 - val_acc: 0.5000
Epoch 2/35
32000/32000 [==============================] - 1s 38us/step - loss: 0.6938 - acc: 0.5012 - val_loss: 0.6920 - val_acc: 0.5000
Epoch 3/35
32000/32000 [==============================] - 1s 38us/step - loss: 0.6912 - acc: 0.5328 - val_loss: 0.6910 - val_acc: 0.5006
Epoch 4/35
32000/32000 [==============================] - 1s 38us/step - loss: 0.6904 - acc: 0.4992 - val_loss: 0.6904 - val_acc: 0.5002
Epoch 5/35
32000/32000 [==============================] - 1s 38us/step - loss: 0.6894 - acc: 0.4988 - val_loss: 0.6887 - val_acc: 0.5001
Epoch 6/35
32000/32000 [==============================] - 1s 39us/step - loss: 0.6872 - acc: 0.4992 - val_loss: 0.6857 - val_acc: 0.5073
Epoch 7/35
32000/32000 [==============================] - 1s 38us/step - loss: 0.6839 - acc: 0.5343 - val_loss: 0.6821 - val_acc

CNN + GloVe

In [ ]:
model_cnn_glove = Sequential()
embedding_layer = Embedding(vocabsize, 100, weights=[embedding_matrix], input_length=maxlength, trainable=False)
model_cnn_glove.add(embedding_layer)
model_cnn_glove.add(Conv1D(512, 5, activation='relu'))
model_cnn_glove.add(GlobalMaxPooling1D())
model_cnn_glove.add(Dense(16, activation='sigmoid'))
model_cnn_glove.add(Dense(1, activation='sigmoid'))
model_cnn_glove.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model_cnn_glove.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          9254700   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 96, 512)           256512    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                8208      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 9,519,437
Trainable params: 264,737
Non-trainable params: 9,254,700
_________________________________________________________________
None


In [ ]:
history_cnn_glove = model_cnn_glove.fit(X_train, Y_train, batch_size=16000, epochs=150, verbose=1, validation_split=0.2)
score = model_cnn_glove.evaluate(X_test, Y_test, verbose=1)
scoretrain = model_cnn_glove.evaluate(X_train, Y_train, verbose=1)
print('Train Accuracy:', scoretrain[1])
print('Test Accuracy:', score[1])

Train on 32000 samples, validate on 8000 samples
Epoch 1/150
32000/32000 [==============================] - 1s 44us/step - loss: 0.7339 - acc: 0.5088 - val_loss: 0.7236 - val_acc: 0.5000
Epoch 2/150
32000/32000 [==============================] - 1s 32us/step - loss: 0.7256 - acc: 0.5012 - val_loss: 0.7098 - val_acc: 0.5000
Epoch 3/150
32000/32000 [==============================] - 1s 32us/step - loss: 0.6984 - acc: 0.5020 - val_loss: 0.6814 - val_acc: 0.6346
Epoch 4/150
32000/32000 [==============================] - 1s 32us/step - loss: 0.6836 - acc: 0.5664 - val_loss: 0.6896 - val_acc: 0.5000
Epoch 5/150
32000/32000 [==============================] - 1s 32us/step - loss: 0.6894 - acc: 0.4988 - val_loss: 0.6860 - val_acc: 0.5001
Epoch 6/150
32000/32000 [==============================] - 1s 32us/step - loss: 0.6832 - acc: 0.5018 - val_loss: 0.6740 - val_acc: 0.5750
Epoch 7/150
32000/32000 [==============================] - 1s 32us/step - loss: 0.6703 - acc: 0.6344 - val_loss: 0.6651 - v

Model 
Pickling


In [ ]:
m1 = 'mlp' + '.joblib'
m2 = 'mlp_glove' + '.joblib'
m3 = 'cnn' + '.joblib'
m4 = 'cnn_glove' + '.joblib'
h1 = 'history_mlp' + '.joblib'
h2 = 'history_mlp_glove' + '.joblib'
h3 = 'history_cnn' + '.joblib'
h4 = 'history_cnn_glove' + '.joblib'

joblib.dump(model_mlp, m1)
joblib.dump(model_mlp_glove, m2)
joblib.dump(model_cnn, m3)
joblib.dump(model_cnn_glove, m4)
joblib.dump(history_mlp, h1)
joblib.dump(history_mlp_glove, h2)
joblib.dump(history_cnn, h3)
joblib.dump(history_cnn_glove, h4)

['history_cnn_glove.joblib']

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': '1rmTGbb19iJn6VbHoHdTYQbK0m0V_EQvP'}]})
file.SetContentFile(m1)
file.Upload() 

file = drive.CreateFile({'parents':[{u'id': '1rmTGbb19iJn6VbHoHdTYQbK0m0V_EQvP'}]})
file.SetContentFile(m2)
file.Upload() 

file = drive.CreateFile({'parents':[{u'id': '1rmTGbb19iJn6VbHoHdTYQbK0m0V_EQvP'}]})
file.SetContentFile(m3)
file.Upload() 

file = drive.CreateFile({'parents':[{u'id': '1rmTGbb19iJn6VbHoHdTYQbK0m0V_EQvP'}]})
file.SetContentFile(m4)
file.Upload() 

file = drive.CreateFile({'parents':[{u'id': '1rmTGbb19iJn6VbHoHdTYQbK0m0V_EQvP'}]})
file.SetContentFile(h1)
file.Upload() 

file = drive.CreateFile({'parents':[{u'id': '1rmTGbb19iJn6VbHoHdTYQbK0m0V_EQvP'}]})
file.SetContentFile(h2)
file.Upload()

file = drive.CreateFile({'parents':[{u'id': '1rmTGbb19iJn6VbHoHdTYQbK0m0V_EQvP'}]})
file.SetContentFile(h3)
file.Upload()

file = drive.CreateFile({'parents':[{u'id': '1rmTGbb19iJn6VbHoHdTYQbK0m0V_EQvP'}]})
file.SetContentFile(h4)
file.Upload()

In [ ]:
# model_mlp = joblib.load('/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/' + m1)
# model_mlp_glove = joblib.load('/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/' + m2)
# model_cnn = joblib.load('/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/' + m3)
# model_cnn_glove = joblib.load('/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/' + m4)
# history_mlp = joblib.load('/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/' + h1)
# history_mlp_glove = joblib.load('/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/' + h2)
# history_cnn = joblib.load('/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/' + h3)
# history_cnn_glove = joblib.load('/content/drive/My Drive/DATASET FOR ML ASSIGNMENT/' + h4)

# Plotting the Graphs

In [ ]:
import matplotlib.pyplot as plt

NN + Self Trained Embedding Layer

In [ ]:
plt.plot(list(range(1, 101)), history_mlp.history['val_loss'], color='green', label='Validation Loss')
plt.plot(list(range(1, 101)), history_mlp.history['loss'], color='brown', label='Training Loss')
plt.plot(list(range(1, 101)), history_mlp.history['val_acc'], color='indigo', label='Validation Accuracy')
plt.plot(list(range(1, 101)), history_mlp.history['acc'], color='orange', label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss/Accuracy')
plt.title('Model Loss & Accuracy (NN with Self Training Embedding Layer)')
plt.legend(loc='best')
plt.show()

NN + GloVe

In [ ]:
plt.plot(list(range(1, 251)), history_mlp_glove.history['val_loss'], color='green', label='Validation Loss')
plt.plot(list(range(1, 251)), history_mlp_glove.history['loss'], color='brown', label='Training Loss')
plt.plot(list(range(1, 251)), history_mlp_glove.history['val_acc'], color='indigo', label='Validation Accuracy')
plt.plot(list(range(1, 251)), history_mlp_glove.history['acc'], color='orange', label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss/Accuracy')
plt.title('Model Loss & Accuracy (NN with GloVe Embedding Layer)')
plt.legend(loc='best')
plt.show()

CNN + Self Trained Embedding Layer

In [ ]:
plt.plot(list(range(1, 36)), history_cnn.history['val_loss'], color='green', label='Validation Loss')
plt.plot(list(range(1, 36)), history_cnn.history['loss'], color='brown', label='Training Loss')
plt.plot(list(range(1, 36)), history_cnn.history['val_acc'], color='indigo', label='Validation Accuracy')
plt.plot(list(range(1, 36)), history_cnn.history['acc'], color='orange', label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss/Accuracy')
plt.title('Model Loss & Accuracy (CNN with Self Training Embedding Layer)')
plt.legend(loc='best')
plt.show()

CNN + GloVe

In [ ]:
plt.plot(list(range(1, 151)), history_cnn_glove.history['val_loss'], color='green', label='Validation Loss')
plt.plot(list(range(1, 151)), history_cnn_glove.history['loss'], color='brown', label='Training Loss')
plt.plot(list(range(1, 151)), history_cnn_glove.history['val_acc'], color='indigo', label='Validation Accuracy')
plt.plot(list(range(1, 151)), history_cnn_glove.history['acc'], color='orange', label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss/Accuracy')
plt.title('Model Loss & Accuracy (CNN with GloVe Embedding Layer)')
plt.legend(loc='best')
plt.show()